In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd
os.chdir("drive/MyDrive/Kolektif Öğrenme/project")
!pwd

/content
/content/drive/MyDrive/Kolektif Öğrenme/project


In [3]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=0952533477e4a3c82c00fc86fc76c20cd3d921a3e1c64d61ef005028eac22789
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=177f7d7b48de4f089fdbe04457e23b11a2e36b7c730f64d1c3afe77c22970af6
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=38f361ac09ab6596e1197c691175fc25aded06abcc78b0487ae15b31bab2b90a
  Sto

In [4]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, RepeatedKFold
import numpy as np
import nltk
from nltk.corpus import twitter_samples
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import GlobalAveragePooling2D, Dense, MaxPooling2D, Flatten, Dropout, Input, Lambda
from keras.models import Model
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
import math
from itertools import combinations as comb


In [6]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [20]:
df = pd.read_json("./Sarcasm_Headlines_Dataset_v2.json", lines= True)


In [21]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [23]:
import tensorflow_hub as hub
from transformers import BertTokenizer, TFBertModel

In [30]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

def encoder(sentences):
  ids = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
    sentence,
    max_length=16,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding['input_ids'])
  return ids

train_sents,test_sents, train_labels, test_labels  = train_test_split(df["headline"],df["is_sarcastic"], test_size=0.2)

train_ids = encoder(train_sents)
test_ids = encoder(test_sents) 

In [31]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [32]:
bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')
input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")  
embedding = bert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l2')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)   
output = tf.keras.layers.Dense(1, activation='sigmoid',kernel_regularizer='l2')(dense)    

model = tf.keras.Model(inputs=[input_word_ids], outputs=output)  

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [34]:
model.compile(tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x = train_ids, y = train_labels, epochs = 5, verbose = 1, batch_size = 64, validation_data = (test_ids, test_labels))

Epoch 1/5


358/358 [==============================] - 111s 260ms/step - loss: 2.4811 - accuracy: 0.5192 - val_loss: 2.3676 - val_accuracy: 0.5168
Epoch 2/5
358/358 [==============================] - 90s 251ms/step - loss: 2.2634 - accuracy: 0.5220 - val_loss: 2.1630 - val_accuracy: 0.5168
Epoch 3/5
358/358 [==============================] - 90s 250ms/step - loss: 2.0694 - accuracy: 0.5255 - val_loss: 1.9798 - val_accuracy: 0.5168
Epoch 4/5
358/358 [==============================] - 90s 250ms/step - loss: 1.8959 - accuracy: 0.5247 - val_loss: 1.8155 - val_accuracy: 0.5168
Epoch 5/5
358/358 [==============================] - 89s 249ms/step - loss: 1.7405 - accuracy: 0.5250 - val_loss: 1.6687 - val_accuracy: 0.5168


In [35]:
bert_encoder = Model(input_word_ids, model.get_layer(list(filter(lambda x: "lambda" in x.name,  model.layers))[0].name).output)
bert_encoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 16)]             0         
                                                                 
 tf_bert_model_2 (TFBertMode  TFBaseModelOutputWithPoo  109482240
 l)                          lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             16, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [36]:
train_embed = bert_encoder.predict(train_ids)
test_embed = bert_encoder.predict(test_ids)

179/179 [==============================] - 9s 53ms/step


In [37]:
def generate_imp_space(X_train, y_train, X_test, imp_feature_size, foz):
    imp_train_data = X_train.values
    imp_test_data = X_test.values
    y_train = pd.DataFrame(y_train)
    d = len(X_train.columns)

    for i in range(0, imp_feature_size*foz):
        Xindis = np.random.permutation(d)
        for j in range(0, d-(foz-1), foz):
            sX = np.random.permutation(1)
            s1 = sX[0]

            s1data = X_train[X_train.index.isin(y_train[y_train == str(s1)].index)]
            s2data = X_train[~X_train.index.isin(y_train[y_train == str(s1)].index)]
            s1data = s1data.iloc[:, Xindis[j:j+(foz)]]
            s2data = s2data.iloc[:, Xindis[j:j+(foz)]]

            s1label = np.ones((s1data.values.shape[0], 1), dtype=int)
            s2label = -1*np.ones((s2data.values.shape[0], 1), dtype=int)
            Wdata = np.concatenate((s1data,s2data))

            Wdata = x2fx(Wdata)
            Wlabel = np.concatenate((s1label, s2label))
            W = np.matmul(np.matmul(np.linalg.pinv(
                np.matmul(Wdata.T, Wdata)), Wdata.T), Wlabel)

            WW = x2fx(X_train.iloc[:, Xindis[j:j+(foz)]].values)
            imp_train_data = np.concatenate(
                (imp_train_data, np.matmul(WW, W)), axis=1)

            TT = x2fx(X_test.iloc[:, Xindis[j:j+(foz)]].values)
            imp_test_data = np.concatenate(
                (imp_test_data, np.matmul(TT, W)), axis=1)

    return imp_train_data, imp_test_data


In [38]:
def x2fx(x, model="linear"):
    linear = np.c_[np.ones(x.shape[0]), x]
    if model == "linear":
        return linear
    if model == "purequadratic":
        return np.c_[linear, x**2]
    interaction = np.hstack([x[:, i]*x[:, j]
                            for i, j in comb(range(x.shape[1]), 2)]).T
    if model == "interaction":
        return np.c_[linear, interaction]
    if model == "quadratic":
        return np.c_[linear, interaction, x**2]

In [40]:
foz = 4
imp_feature_size = 1
n_estimators = 5
imp_acc = []
rfc_acc = []
estimators_imp = []
estimators = []
train_embed = pd.DataFrame(train_embed)
test_embed=pd.DataFrame(test_embed)
d = len(train_embed.columns)

for i in range(n_estimators):
    imp_tr, imp_ts = generate_imp_space(train_embed, train_labels, test_embed, imp_feature_size, foz)
    imp_d = imp_tr.shape[1]

    imp_sel_d = 2 * round(math.log2(imp_d))
    sel_d = 2*round(math.log2(d))

    imp_rfc = RandomForestClassifier(max_features=imp_sel_d, n_estimators=50, random_state=42)
    imp_rfc.fit(imp_tr, train_labels)
    estimators_imp.append(("imp_rfc"+str(i), imp_rfc))

    rfc = RandomForestClassifier(max_features=sel_d, n_estimators=50, random_state=42)
    rfc.fit(train_embed, train_labels)
    estimators.append(("rfc"+str(i), rfc))


voting_imp = VotingClassifier(estimators=estimators_imp)
voting_imp.fit(imp_tr, train_labels)

voting_rfc = VotingClassifier(estimators=estimators)
voting_rfc.fit(train_embed, train_labels)

imp_acc.append(voting_imp.score(imp_ts, test_labels))
rfc_acc.append(voting_rfc.score(test_embed, test_labels))


In [41]:
imp_acc

[0.5435010482180294]

In [42]:
rfc_acc

[0.5414046121593291]

In [43]:
import xgboost as xgb

In [44]:
foz = 4
imp_feature_size = 1
n_estimators = 5
imp_xgbc_acc = []
xgbc_acc = []
estimators_imp = []
estimators = []
train_embed = pd.DataFrame(train_embed)
test_embed=pd.DataFrame(test_embed)
d = len(train_embed.columns)

for i in range(n_estimators):
    imp_tr, imp_ts = generate_imp_space(train_embed, train_labels, test_embed, imWp_feature_size, foz)
    imp_d = imp_tr.shape[1]

    imp_sel_d = 2 * round(math.log2(imp_d))
    sel_d = 2*round(math.log2(d))

    imp_xgbc = xgb.XGBClassifier(max_features=imp_sel_d, n_estimators=50, random_state=42)
    imp_xgbc.fit(imp_tr, train_labels)
    estimators_imp.append(("imp_xgbc"+str(i), imp_rfc))

    xgbc = xgb.XGBClassifier(max_features=sel_d, n_estimators=50, random_state=42)
    xgbc.fit(train_embed, train_labels)
    estimators.append(("xgbc"+str(i), rfc))


voting_imp = VotingClassifier(estimators=estimators_imp)
voting_imp.fit(imp_tr, train_labels)

voting_xgb = VotingClassifier(estimators=estimators)
voting_xgb.fit(train_embed, train_labels)

imp_xgbc_acc = voting_imp.score(imp_ts, test_labels)
xgbc_acc = voting_xgb.score(test_embed, test_labels)


In [45]:
imp_xgbc_acc

0.5400069881201957

In [46]:
xgbc_acc

0.5414046121593291